In [1]:
import pandas as pd
import stock_data as stock
from tqdm import tqdm

In [2]:
start = "2010-01-01"
end = "2021-12-31"
mode = "all"

scrapy = stock.Scrapy()
df0 = scrapy.get_price(start, end, mode)

------------------------------ Get ticker. ------------------------------
------------------------------ Get price. ------------------------------
[*********************100%***********************]  1767 of 1767 completed

7 Failed downloads:
- 5244.TW: Data doesn't exist for startDate = 1262275200, endDate = 1640880000
- 6719.TW: Data doesn't exist for startDate = 1262275200, endDate = 1640880000
- 1752.TW: Data doesn't exist for startDate = 1262275200, endDate = 1640880000
- 3715.TW: Data doesn't exist for startDate = 1262275200, endDate = 1640880000
- 6807.TW: Data doesn't exist for startDate = 1262275200, endDate = 1640880000
- 3592.TW: Data doesn't exist for startDate = 1262275200, endDate = 1640880000
- 3652.TW: No data found for this date range, symbol may be delisted
------------------------------ Clean data. ------------------------------


100%|██████████| 1767/1767 [02:12<00:00, 13.31it/s]


In [3]:
df0["Date"] = df0["Date"].apply(lambda X: X.date())
df0["Year"] = df0["Date"].apply(lambda X: X.year)
g = df0.groupby(["Symbol", "Year"])
cat = g.size().keys()

data = []
for i in tqdm(range(len(cat))):
    sample = []
    df1 = g.get_group(cat[i])
    min_ = df1.loc[df1["Close"].idxmin()][["Date", "Close"]].values
    max_ = df1.loc[df1["Close"].idxmax()][["Date", "Close"]].values
    return_ = (df1["Close"].max() - df1["Close"].min()) / df1["Close"].min()

    sample.extend(list(cat[i]))
    sample.extend(min_)
    sample.extend(max_)
    sample.append(return_)
    data.append(sample)
df = pd.DataFrame(data, columns = ["symbol", "year", "min_date", "min_price", "max_date", "max_price", "return"])
df

100%|██████████| 18877/18877 [00:41<00:00, 457.62it/s]


,symbol,year,min_date,min_price,max_date,max_price,return
0,1101.TW,2010,2010-05-07,18.45,2010-01-06,26.33,0.427100
1,1101.TW,2011,2011-09-26,21.52,2011-07-22,35.60,0.654275
2,1101.TW,2012,2012-05-18,23.54,2012-12-07,28.98,0.231096
3,1101.TW,2013,2013-06-25,24.83,2013-11-29,34.46,0.387837
4,1101.TW,2014,2014-12-17,30.56,2014-08-22,36.45,0.192736
...,...,...,...,...,...,...,...
18872,9962.TWO,2017,2017-01-09,9.92,2017-03-14,11.55,0.164315
18873,9962.TWO,2018,2018-01-23,10.15,2018-06-29,13.10,0.290640
18874,9962.TWO,2019,2019-12-03,9.96,2019-04-08,12.15,0.219880
18875,9962.TWO,2020,2020-03-19,6.48,2020-01-02,10.10,0.558642


In [4]:
df["market"] = df["symbol"].apply(lambda X: X.split(".")[-1])
df = df.query("min_price >= 5")
df = df.reset_index(drop = True)
df

,symbol,year,min_date,min_price,max_date,max_price,return,market
0,1101.TW,2010,2010-05-07,18.45,2010-01-06,26.33,0.427100,TW
1,1101.TW,2011,2011-09-26,21.52,2011-07-22,35.60,0.654275,TW
2,1101.TW,2012,2012-05-18,23.54,2012-12-07,28.98,0.231096,TW
3,1101.TW,2013,2013-06-25,24.83,2013-11-29,34.46,0.387837,TW
4,1101.TW,2014,2014-12-17,30.56,2014-08-22,36.45,0.192736,TW
...,...,...,...,...,...,...,...,...
18393,9962.TWO,2017,2017-01-09,9.92,2017-03-14,11.55,0.164315,TWO
18394,9962.TWO,2018,2018-01-23,10.15,2018-06-29,13.10,0.290640,TWO
18395,9962.TWO,2019,2019-12-03,9.96,2019-04-08,12.15,0.219880,TWO
18396,9962.TWO,2020,2020-03-19,6.48,2020-01-02,10.10,0.558642,TWO


In [5]:
df = df.query("max_date > min_date")
df = df.reset_index(drop = True)
df

,symbol,year,min_date,min_price,max_date,max_price,return,market
0,1101.TW,2012,2012-05-18,23.54,2012-12-07,28.98,0.231096,TW
1,1101.TW,2013,2013-06-25,24.83,2013-11-29,34.46,0.387837,TW
2,1101.TW,2016,2016-01-26,18.76,2016-11-10,28.06,0.495736,TW
3,1101.TW,2018,2018-02-06,26.11,2018-08-29,35.12,0.345079,TW
4,1101.TW,2019,2019-01-03,28.44,2019-12-31,37.84,0.330520,TW
...,...,...,...,...,...,...,...,...
8966,9962.TWO,2010,2010-02-03,10.73,2010-12-27,16.96,0.580615,TWO
8967,9962.TWO,2016,2016-01-14,7.01,2016-04-22,10.45,0.490728,TWO
8968,9962.TWO,2017,2017-01-09,9.92,2017-03-14,11.55,0.164315,TWO
8969,9962.TWO,2018,2018-01-23,10.15,2018-06-29,13.10,0.290640,TWO


In [6]:
g = df.groupby(["year", "market"])
cat = g.size().keys()

df2 = pd.DataFrame()
for i in tqdm(range(len(cat))):
    df1 = g.get_group(cat[i])
    df2 = pd.concat([df2, df1.nlargest(50, "return")], ignore_index = True)
df2 = df2.sort_values(["year", "market", "return"])
df2 = df2[['year', 'market', 'symbol', 'min_date', 'min_price', 'max_date', 'max_price', 'return']]
df2["return"] = df2["return"].round(4)
df2

100%|██████████| 24/24 [00:00<00:00, 258.09it/s]


,year,market,symbol,min_date,min_price,max_date,max_price,return
49,2010,TW,1456.TW,2010-05-06,7.33,2010-12-23,16.76,1.2865
48,2010,TW,9911.TW,2010-02-05,16.76,2010-08-17,38.48,1.2959
47,2010,TW,2201.TW,2010-02-08,44.12,2010-10-06,101.45,1.2994
46,2010,TW,3060.TW,2010-01-07,41.60,2010-12-28,96.70,1.3245
45,2010,TW,8210.TW,2010-02-08,20.47,2010-12-17,47.71,1.3307
...,...,...,...,...,...,...,...,...
1154,2021,TWO,3169.TWO,2021-01-20,54.40,2021-12-06,290.00,4.3309
1153,2021,TWO,6446.TWO,2021-10-07,70.30,2021-12-13,396.00,4.6330
1152,2021,TWO,6485.TWO,2021-01-12,32.70,2021-08-05,186.50,4.7034
1151,2021,TWO,6763.TWO,2021-01-19,219.28,2021-09-08,1333.33,5.0805
